In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/evaluate/repeat10/Prediction/four/less_data/probability.csv", header=None)
result = data.groupby('y_true')['y_pred'].value_counts().unstack(fill_value=0)
result

FileNotFoundError: [Errno 2] No such file or directory: '/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ATM_TCR/evaluate/repeat10/Prediction/four/less_data/probability.csv'

In [ ]:
data=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/NetTCR/evaluate/repeat10/Prediction/repeat/withCF/1_threeprobability.csv")
result = data.groupby('y_true')['y_pred'].value_counts().unstack(fill_value=0)
result

In [3]:
import pandas as pd

def analyze_tcr(df, affinity, model_prefix):
    df_filtered = df[df['Affinity'] == affinity]
    count_cf_ending = df_filtered['CDR3B'].apply(lambda x: x.startswith('C') and x.endswith('F')).sum()
    count_not_cf_ending = len(df_filtered) - count_cf_ending
    unique_epitopes = df_filtered['Epitope'].nunique()

    model_name = f"{model_prefix}_{'pos' if affinity == 1 else 'neg'}"
    result_df = pd.DataFrame({
        'Model Name': [model_name],
        'Non-CF TCR': [count_not_cf_ending],
        'CF TCR': [count_cf_ending],
        'All TCR': [count_cf_ending+count_not_cf_ending],
        'Unique Epitopes': [unique_epitopes]
    })
    return result_df

# DLpTCR

In [4]:
import pandas as pd
DLpTCR=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/DLpTCR/data/TCRB_train.csv")
DLpTCR=DLpTCR[['Class_label','CDR3','Epitope']]
DLpTCR.rename(columns={'Class_label':'Affinity','CDR3':'CDR3B'},inplace=True)

In [5]:
DLpTCR_pos = analyze_tcr(DLpTCR, 1, "DLpTCR")
DLpTCR_neg = analyze_tcr(DLpTCR, 0, "DLpTCR")


# NetTCR

In [6]:
NetTCR99=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/NetTCR/data/train_beta_99.csv")
NetTCR99=NetTCR99[['CDR3B','Epitope','Affinity']]
NetTCR99
NetTCR99_pos = analyze_tcr(NetTCR99, 1, "NetTCR")
NetTCR99_neg = analyze_tcr(NetTCR99, 0, "NetTCR")

In [7]:
NetTCR99

,CDR3B,Epitope,Affinity
0,ASRPGLMGGRPEQY,LLFGYPVYV,1
1,ASSAGLFPEQF,LLFGYPVYV,1
2,ASSQVSGLAGGQNTGELF,GLCTLVAML,1
3,ASSQDGLAGGAFNEQF,GLCTLVAML,1
4,ASSLVKLGIALIETQY,GLCTLVAML,1
...,...,...,...
55219,ASIRGTSGASGQETQY,LLFGYPVYV,0
55220,ASSLRGRGEGYT,LLFGYPVYV,0
55221,SAPGTGNQETQY,LLFGYPVYV,0
55222,SVEASGRGDNEQF,LLFGYPVYV,0


# tcr_bert

In [8]:
tcr_bert=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/data/pird/fileter_pird.csv", encoding='latin1')
tcr_bert.rename(columns={'Antigen.sequence': 'Epitope','CDR3.beta.aa':'CDR3B'}, inplace=True)
tcr_epi=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/tcr_bert/data/pird/files.csv",header=None)
tcr_epi.columns = ['Epitope']
tcr_epi['Epitope'] = tcr_epi['Epitope'].str.replace('.txt', '')
tcr_bert = pd.merge(tcr_bert, tcr_epi, on='Epitope', how='inner')
tcr_bert=tcr_bert[['Epitope','CDR3B']]
tcr_bert['Affinity']= 1
tcr_bert_pos = analyze_tcr(tcr_bert, 1, "tcr_bert")
tcr_bert_pos

,Model Name,Non-CF TCR,CF TCR,All TCR,Unique Epitopes
0,tcr_bert_pos,2,6150,6152,43


In [20]:
filtered_df = tcr_bert[(tcr_bert['Epitope'].isin(['GILGFVFTL', 'GLCTLVAML', 'NLVPMVATV'])) & (tcr_bert['Affinity'] == 1)]
counts = filtered_df.groupby('Epitope')['CDR3B'].nunique()
counts

Epitope
GILGFVFTL     149
GLCTLVAML     122
NLVPMVATV    4053
Name: CDR3B, dtype: int64

# ERGO

In [9]:
import pickle
with open("/home/luyanping/data/TCR_epitope_prediction/ERGO/ERGO/models/lstm_vdjdb_train1.pickle", "rb") as file:
    ERGOvdj_lstm = pickle.load(file)
ERGOvdj_lstm = pd.DataFrame(ERGOvdj_lstm, columns=['CDR3B', 'Epitope', 'A'])
ERGOvdj_lstm['Epitope'] = ERGOvdj_lstm['Epitope'].apply(lambda x: x[0])
ERGOvdj_lstm['A'] = ERGOvdj_lstm['A'].replace({'n': 0, 'p': 1})
ERGOvdj_lstm.rename(columns={'A':'Affinity'},inplace=True)
ERGOvdj_lstm

with open("/home/luyanping/data/TCR_epitope_prediction/ERGO/ERGO/models/lstm_mcpas_train1.pickle", "rb") as file:
    ERGOmc_lstm = pickle.load(file)
ERGOmc_lstm = pd.DataFrame(ERGOmc_lstm, columns=['CDR3B', 'Epitope', 'A'])
ERGOmc_lstm['Epitope'] = ERGOmc_lstm['Epitope'].apply(lambda x: x[0])
ERGOmc_lstm['A'] = ERGOmc_lstm['A'].replace({'n': 0, 'p': 1})
ERGOmc_lstm.rename(columns={'A':'Affinity'},inplace=True)
ERGOmc_lstm


with open("/home/luyanping/data/TCR_epitope_prediction/ERGO/ERGO/models/ae_vdjdb_train1.pickle", "rb") as file:
    ERGOvdj_ae = pickle.load(file)
ERGOvdj_ae = pd.DataFrame(ERGOvdj_ae, columns=['CDR3B', 'Epitope', 'A'])
ERGOvdj_ae['Epitope'] = ERGOvdj_ae['Epitope'].apply(lambda x: x[0])
ERGOvdj_ae['A'] = ERGOvdj_ae['A'].replace({'n': 0, 'p': 1})
ERGOvdj_ae.rename(columns={'A':'Affinity'},inplace=True)
ERGOvdj_ae


with open("/home/luyanping/data/TCR_epitope_prediction/ERGO/ERGO/models/ae_mcpas_train1.pickle", "rb") as file:
    ERGOmc_ae = pickle.load(file)
ERGOmc_ae = pd.DataFrame(ERGOmc_ae, columns=['CDR3B', 'Epitope', 'A'])
ERGOmc_ae['Epitope'] = ERGOmc_ae['Epitope'].apply(lambda x: x[0])
ERGOmc_ae['A'] = ERGOmc_ae['A'].replace({'n': 0, 'p': 1})
ERGOmc_ae.rename(columns={'A':'Affinity'},inplace=True)
ERGOmc_ae


FileNotFoundError: [Errno 2] No such file or directory: '/home/luyanping/data/TCR_epitope_prediction/ERGO/ERGO/models/lstm_vdjdb_train1.pickle'

In [10]:
ERGOvdj_lstm_pos = analyze_tcr(ERGOvdj_lstm, 1, "ERGOvdj_lstm")
ERGOmc_lstm_pos = analyze_tcr(ERGOmc_lstm, 1, "ERGOmc_lstm")
ERGOvdj_ae_pos = analyze_tcr(ERGOvdj_ae, 1, "ERGOvdj_ae")
ERGOmc_ae_pos = analyze_tcr(ERGOmc_ae, 1, "ERGOmc_ae")

NameError: name 'ERGOvdj_lstm' is not defined

In [16]:
ERGOvdj_lstm_neg = analyze_tcr(ERGOvdj_lstm, 0, "ERGOvdj_lstm")
ERGOmc_lstm_neg = analyze_tcr(ERGOmc_lstm, 0, "ERGOmc_lstm")
ERGOvdj_ae_neg = analyze_tcr(ERGOvdj_ae, 0, "ERGOvdj_ae")
ERGOmc_ae_neg = analyze_tcr(ERGOmc_ae, 0, "ERGOmc_ae")

# epiTCR

In [11]:
epiTCR=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/epiTCR/data/splitData/withoutMHC/train/train.csv")
epiTCR.rename(columns={'CDR3b':'CDR3B','epitope':'Epitope','binder':'Affinity'},inplace=True)

In [12]:
epiTCR_pos = analyze_tcr(epiTCR, 1, "epiTCR")
epiTCR_neg = analyze_tcr(epiTCR, 0, "epiTCR")

# TEINet

In [13]:
import pandas as pd
TEINet_Art=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEINet/data_yun/processed_data/whole_data.csv")
TEINet_Art.rename(columns={'CDR3.beta':'CDR3B'},inplace=True)
TEINet_Art=TEINet_Art[['CDR3B','Epitope']]
TEINet_Art['Affinity']=1
TEINet_Art_pos = analyze_tcr(TEINet_Art, 1, "TEINet_Art")
TEINet_Art_pos

,Model Name,Non-CF TCR,CF TCR,All TCR,Unique Epitopes
0,TEINet_Art_pos,0,44682,44682,180


In [14]:
TEINet_large=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEINet/data_yun/5fold/combined.csv")
epitope_counts = TEINet_large['Epitope'].value_counts()
TEINet_large = TEINet_large[TEINet_large['Epitope'].isin(epitope_counts[epitope_counts > 10].index)]
TEINet_large.rename(columns={'CDR3.beta':'CDR3B'},inplace=True)
TEINet_large['Affinity']= 1
TEINet_large=TEINet_large[['CDR3B','Epitope','Affinity']]
TEINet_large_pos = analyze_tcr(TEINet_large, 1, "TEINet_large")
TEINet_large_pos

,Model Name,Non-CF TCR,CF TCR,All TCR,Unique Epitopes
0,TEINet_large_pos,0,115209,115209,356


In [15]:
TEINet_large

,CDR3B,Epitope,Affinity
0,CASSESAGGYYNEQF,IVTDFSVIK,1
1,CASSLASATGELF,AVFDRKSDAK,1
2,CASSLSRGLLNEAF,IVTDFSVIK,1
3,CASSLTSAAGELF,AVFDRKSDAK,1
4,CASSPTSAAGELF,AVFDRKSDAK,1
...,...,...,...
118155,CTIGRTNTEAFF,SPRWYFYYL,1
118158,CASSRDYNEQFF,YLQPRTFLL,1
118160,CSVLGGVNEQFF,SPRWYFYYL,1
118162,CSVMDSLNTGELFF,YLQPRTFLL,1


# ATM_TCR

In [18]:
ATM_TCR=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/ATM_TCR/data/combined_dataset.csv",header=None)
ATM_TCR.columns =['Epitope', 'CDR3B', 'Affinity']

FileNotFoundError: [Errno 2] No such file or directory: '/home/luyanping/data/TCR_epitope_prediction/ATM_TCR/data/combined_dataset.csv'

In [17]:
ATM_TCR_pos = analyze_tcr(ATM_TCR, 1, "ATM_TCR")
ATM_TCR_neg = analyze_tcr(ATM_TCR, 0, "ATM_TCR")

NameError: name 'ATM_TCR' is not defined

# AttnTAP

In [24]:
AttnTAP_VDJdb=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/AttnTAP/Data/VDJdb/VDJdb_train.csv")
AttnTAP_VDJdb.rename(columns={'tcr':'CDR3B','antigen':'Epitope','label':'Affinity'},inplace=True)

AttnTAP_McPAS=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/AttnTAP/Data/McPAS/McPAS_train.csv")
AttnTAP_McPAS.rename(columns={'tcr':'CDR3B','antigen':'Epitope','label':'Affinity'},inplace=True)


In [25]:
AttnTAP_VDJdb_pos = analyze_tcr(AttnTAP_VDJdb, 1, "AttnTAP_VDJdb")
AttnTAP_VDJdb_neg = analyze_tcr(AttnTAP_VDJdb, 0, "AttnTAP_VDJdb")

In [26]:
AttnTAP_McPAS_pos = analyze_tcr(AttnTAP_McPAS, 1, "AttnTAP_McPAS")
AttnTAP_McPAS_neg = analyze_tcr(AttnTAP_McPAS, 0, "AttnTAP_McPAS")

# TPBTE

In [27]:
import os
import pandas as pd

folder_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TPBTE/VDJdb/multi/"
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
merged_data = pd.DataFrame()
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path,header=None)
    TPBTE_VDJdb = pd.concat([merged_data, df])
TPBTE_VDJdb.drop_duplicates(inplace=True)
TPBTE_VDJdb.columns =['CDR3B','Epitope', 'Affinity']



folder_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TPBTE/McPAS/multi/"
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
TPBTE_McPAS = pd.DataFrame()
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path,header=None)
    TPBTE_McPAS = pd.concat([merged_data, df])
TPBTE_McPAS.drop_duplicates(inplace=True)
TPBTE_McPAS.columns =['CDR3B','Epitope', 'Affinity']

In [28]:
TPBTE_VDJdb_pos = analyze_tcr(TPBTE_VDJdb, 1, "TPBTE_VDJdb")
TPBTE_VDJdb_neg = analyze_tcr(TPBTE_VDJdb, 0, "TPBTE_VDJdb")

In [29]:
TPBTE_McPAS_pos = analyze_tcr(TPBTE_McPAS, 1, "TPBTE_McPAS")
TPBTE_McPAS_neg = analyze_tcr(TPBTE_McPAS, 0, "TPBTE_McPAS")

# SETE

In [30]:
SETE=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/SETE/data/epitope_tcr_pairs.csv")
SETE.rename(columns={'epitope':'Epitope','cdr3b':'CDR3B'},inplace=True)
SETE=SETE[['Epitope','CDR3B']]
SETE['Affinity']=1
SETE

,Epitope,CDR3B,Affinity
0,LLWNGPMAV,CASSEATGASYEQYF,1
1,LLWNGPMAV,CASSEGGQAYNEQFF,1
2,LLWNGPMAV,CASSVATEGYGYTF,1
3,LLWNGPMAV,CASSQGTISYEQYF,1
4,LLWNGPMAV,CASSVGGVTYNEQFF,1
...,...,...,...
3028,GTSGSPIINR,CASSLQPSGTSFYNEQFF,1
3029,GTSGSPIINR,CASSLGTREAFF,1
3030,GTSGSPIINR,CASTTGGGGYEQYF,1
3031,GTSGSPIINR,CASSLGQGSYEQYF,1


In [31]:
SETE_pos = analyze_tcr(SETE, 1, "SETE")

# TCRGP

In [32]:
folder_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TCRGP/training_data/examples/"
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
dfs = []
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path)
    dfs.append(df)
TCRGP = pd.concat(dfs, ignore_index=True) 
TCRGP.rename(columns={'epitope':'Epitope','cdr3b':'CDR3B'},inplace=True)
TCRGP=TCRGP[['Epitope','CDR3B']]
TCRGP['Affinity'] = TCRGP['Epitope'].apply(lambda x: 0 if x == 'none' else 1)
TCRGP=TCRGP[TCRGP['Affinity']==1]
TCRGP

,Epitope,CDR3B,Affinity
0,GTSGSPIVNR,CASSLGSYEQYF,1
1,GTSGSPIVNR,CASSLVGGTPSYNEQFF,1
2,GTSGSPIVNR,CASSLFAGGPDTQYF,1
3,GTSGSPIVNR,CASSWGAAEAFF,1
4,GTSGSPIVNR,CASSLVIPTGWRDNEQFF,1
...,...,...,...
5909,ATDALMTGY,CAGNNRDSSTDTQYF,1
5910,ATDALMTGY,CSARPIDSSNQPQHF,1
5911,ATDALMTGY,CAISESTAGSQPQHF,1
5912,ATDALMTGY,CASSGTRQISGTEAFF,1


In [33]:
TCRGP_pos = analyze_tcr(TCRGP, 1, "TCRGP")

# TEIM

In [34]:
TEIM1=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/TEIM/data/binding_data/mcpas_seen.tsv", sep='\t')
TEIM2=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/TEIM/data/binding_data/ncov_unseen.tsv", sep='\t')
TEIM3=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/TEIM/data/binding_data/mcpas_unseen.tsv", sep='\t')
TEIM4=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/TEIM/data/binding_data/vdj_all.tsv", sep='\t')
TEIM = pd.concat([TEIM1, TEIM2, TEIM3, TEIM4], axis=0)
TEIM=TEIM[['epitope','cdr3','label']]
TEIM.rename(columns={'epitope':'Epitope','cdr3':'CDR3B','label':'Affinity'},inplace=True)
TEIM_pos = analyze_tcr(TEIM, 1, "TEIM")
TEIM_pos

,Model Name,Non-CF TCR,CF TCR,All TCR,Unique Epitopes
0,TEIM_pos,94,45203,45297,355


# PanPep

In [35]:
PanPep= pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/pan_peptide/PanPep/Data/base_dataset.csv")
PanPep.rename(columns={'peptide':'Epitope','binding_TCR':'CDR3B','label':'Affinity'},inplace=True)
PanPep_pos = analyze_tcr(PanPep, 1, "PanPep")
PanPep_pos

,Model Name,Non-CF TCR,CF TCR,All TCR,Unique Epitopes
0,PanPep_pos,1229,30851,32080,699


# iTCep

In [36]:
iTCep= pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/iTCep/data/dataset/training_dataset.csv")
iTCep.rename(columns={'peptide':'Epitope','CDR3':'CDR3B'},inplace=True)
iTCep.rename(columns={'label':'Affinity'},inplace=True)
iTCep

,Epitope,CDR3B,Affinity
0,NLVPMVATV,CASSQWSNEKLFF,1
1,LPRRSGAAGA,CASSGDSDEQYF,1
2,EEYLKAWTF,CATGPGQGYYGYTF,0
3,NLVPMVATV,CSVEGVRKTNTGELFF,1
4,QYIKWPWYI,CASSPRTGVDEQFF,0
...,...,...,...
17209,GLCTLVAML,CSARGQATTEKLFF,1
17210,KRWIILGLNK,CASSHGDYEQYF,1
17211,SFHSLHLLF,CASRPNRDSNYGYTF,1
17212,NLVPMVATV,CASSYGPEAHNEQFF,1


In [37]:
iTCep_pos = analyze_tcr(iTCep, 1, "iTCep")

# ImRex

In [38]:
ImRex=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/ImRex/models/pretrained/2020-07-24_19-18-39_trbmhcidown-shuffle-padded-b32-lre4-reg001/iteration_no_validation/train.csv")
ImRex = pd.DataFrame(ImRex['y;cdr3;antigen.epitope'].str.split(';', expand=True))
ImRex.columns = ['Affinity', 'CDR3B', 'Epitope']   
ImRex['Affinity'] = pd.to_numeric(ImRex['Affinity'])

ImRex_pos = analyze_tcr(ImRex, 1, "ImRex")
ImRex_neg = analyze_tcr(ImRex, 0, "ImRex")

# merged

In [39]:
pos=[DLpTCR_pos,NetTCR99_pos,tcr_bert_pos,ERGOvdj_lstm_pos,ERGOmc_lstm_pos,ERGOvdj_ae_pos,ERGOmc_ae_pos,epiTCR_pos,TEINet_Art_pos,TEINet_large_pos,ATM_TCR_pos,AttnTAP_VDJdb_pos,
   AttnTAP_McPAS_pos ,TPBTE_VDJdb_pos,TPBTE_McPAS_pos,TEIM_pos,ImRex_pos,TCRGP_pos]
merged_pos = pd.concat(pos, ignore_index=True)
merged_pos.columns = ['Model', 'Non_CF_TCR_pos', 'CF_TCR_pos', 'Total_TCR_pos', 'Unique_Epitopes_pos']
merged_pos['Model'] = merged_pos['Model'].str.replace('_pos', '')
merged_pos

,Model,Non_CF_TCR_pos,CF_TCR_pos,Total_TCR_pos,Unique_Epitopes_pos
0,DLpTCR,1,5709,5710,278
1,NetTCR,9204,0,9204,7
2,tcr_bert,2,6150,6152,43
3,ERGOvdj_lstm,0,34963,34963,205
4,ERGOmc_lstm,165,11109,11274,302
5,ERGOvdj_ae,0,34952,34952,200
6,ERGOmc_ae,153,11011,11164,316
7,epiTCR,6684,0,6684,487
8,TEINet_Art,0,44682,44682,180
9,TEINet_large,0,115209,115209,356


In [40]:
neg=[DLpTCR_neg,NetTCR99_neg,ERGOvdj_lstm_neg,ERGOmc_lstm_neg,ERGOvdj_ae_neg,ERGOmc_ae_neg,epiTCR_neg,ATM_TCR_neg,AttnTAP_VDJdb_neg,AttnTAP_McPAS_neg,
     TPBTE_VDJdb_neg,TPBTE_McPAS_neg,ImRex_neg]
merged_neg = pd.concat(neg, ignore_index=True)
merged_neg.columns = ['Model', 'Non_CF_TCR_neg', 'CF_TCR_neg', 'Total_TCR_neg', 'Unique_Epitopes_neg']
merged_neg['Model'] = merged_neg['Model'].str.replace('_neg', '')
merged_neg


,Model,Non_CF_TCR_neg,CF_TCR_neg,Total_TCR_neg,Unique_Epitopes_neg
0,DLpTCR,181,5529,5710,282
1,NetTCR,46020,0,46020,7
2,ERGOvdj_lstm,0,174816,174816,195
3,ERGOmc_lstm,949,55423,56372,270
4,ERGOvdj_ae,0,174761,174761,197
5,ERGOmc_ae,822,54999,55821,292
6,epiTCR,314672,9,314681,50
7,ATM_TCR,12936,115206,128142,931
8,AttnTAP_VDJdb,0,30507,30507,56
9,AttnTAP_McPAS,0,7677,7677,25


In [41]:
merged_pos = merged_pos.applymap(str)
merged_neg = merged_neg.applymap(str)
merged_all = pd.merge(merged_pos, merged_neg, on='Model', how='left')
merged_all = merged_all.fillna('-')
merged_all.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/fig/0614/data/prediction/Raw_model_training_dataset.csv")

# 统计表位对应TCR数量

In [43]:
import pandas as pd
def filter_and_count_cdr3b(df):
    filtered_df = df[(df['Epitope'].isin(['GILGFVFTL', 'GLCTLVAML', 'NLVPMVATV'])) & (df['Affinity'] == 1)]
    counts = filtered_df.groupby('Epitope')['CDR3B'].nunique()
    return counts

datasets = [DLpTCR, ERGOvdj_lstm, ERGOmc_lstm, ERGOvdj_ae, ERGOmc_ae, epiTCR, 
    TEINet_Art, TEINet_large, ATM_TCR, AttnTAP_VDJdb, AttnTAP_McPAS, 
    TPBTE_VDJdb, TPBTE_McPAS, SETE, TCRGP, TEIM, ImRex, NetTCR99]

counts_per_dataset = [filter_and_count_cdr3b(df) for df in datasets]
combined_counts = pd.concat(counts_per_dataset, axis=1)
combined_counts.columns = ['DLpTCR', 'ERGOvdj_lstm', 'ERGOmc_lstm', 'ERGOvdj_ae', 'ERGOmc_ae', 'epiTCR', 
                           'TEINet_Art', 'TEINet_large', 'ATM_TCR', 'AttnTAP_VDJdb', 'AttnTAP_McPAS', 
                           'TPBTE_VDJdb', 'TPBTE_McPAS', 'SETE', 'TCRGP', 'TEIM', 'ImRex', 'NetTCR99']
combined_counts = combined_counts.T 
combined_counts.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/fig/0614/data/prediction/three_epitope_tcr.csv")

# 数据集合并

In [44]:
import numpy as np
from functools import reduce
datasets = [ DLpTCR, ERGOvdj_lstm, ERGOmc_lstm, ERGOvdj_ae, ERGOmc_ae, epiTCR, 
    TEINet_Art, TEINet_large, ATM_TCR, AttnTAP_VDJdb, AttnTAP_McPAS, 
    TPBTE_VDJdb, TPBTE_McPAS, SETE, TCRGP, TEIM, ImRex, NetTCR99]
unique_epitopes = [pd.unique(dataset['Epitope']) for dataset in datasets]
common_elements = reduce(np.intersect1d, unique_epitopes)
common_elements

array(['GILGFVFTL', 'GLCTLVAML', 'NLVPMVATV'], dtype=object)

In [45]:
def filter_dataframe(df):
    return df[df['Epitope'].isin(['GILGFVFTL', 'GLCTLVAML', 'NLVPMVATV'])]
datasets = [ DLpTCR, ERGOvdj_lstm, ERGOmc_lstm, ERGOvdj_ae, ERGOmc_ae, epiTCR, 
    TEINet_Art, TEINet_large, ATM_TCR, AttnTAP_VDJdb, AttnTAP_McPAS, 
    TPBTE_VDJdb, TPBTE_McPAS, SETE, TCRGP, TEIM, ImRex, NetTCR99]

filtered_dataframes = [filter_dataframe(df) for df in datasets]
combined_df = pd.concat(filtered_dataframes, ignore_index=True)

combined_df = combined_df.drop_duplicates()
pos_combined= combined_df[combined_df['Affinity'] == 1]
neg_combined = combined_df[combined_df['Affinity'] == 0]

# 所有数据集

In [54]:
file_path ="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/IEDB/receptor_table_export_0714.xlsx"
IEDB = pd.read_excel(file_path)

In [93]:
IEDB_new=IEDB[['Epitope','CDR3_Curated_beita','Source Organism']]
IEDB_new['MHC']='MHCI'
IEDB_new['database']='IEDB'
IEDB_df = IEDB_new.dropna(subset=['CDR3_Curated_beita'])
column_mapping = {
    "CDR3_Curated_beita": "CDR3B",
    "Source Organism": "antigen_species"}
IEDB_df=IEDB_df.rename(columns=column_mapping)
IEDB_df['CDR3B'] = IEDB_df['CDR3B'].apply(lambda x: ('C' if not x.startswith('C') else '') + x + ('F' if not x.endswith('F') else ''))
IEDB_df

/tmp/ipykernel_43612/3182142058.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IEDB_new['MHC']='MHCI'
/tmp/ipykernel_43612/3182142058.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  IEDB_new['database']='IEDB'


,Epitope,CDR3B,antigen_species,MHC,database
34,CINGVCWTV,CASSQGGDRGDPGDGYTF,Hepatitis C virus,MHCI,IEDB
68,LLFGYPVYV,CASRPGLAGGRPEQYF,Human T-cell leukemia virus type I (Human T ce...,MHCI,IEDB
70,LLFGYPVYV,CASRPGLMSAQPEQYF,Human T-cell leukemia virus type I (Human T ce...,MHCI,IEDB
96,SLLMWITQC,CASSYVGNTGELFF,Homo sapiens (human),MHCI,IEDB
139,GLCTLVAML,CSARDGTGNGYTF,Human herpesvirus 4 (Epstein Barr virus),MHCI,IEDB
...,...,...,...,...,...
213181,NQKLIANQF,CASSYSNRGSTDTQYF,SARS-CoV2,MHCI,IEDB
213182,NQKLIANQF,CASSYSSLGSTDTQYF,SARS-CoV2,MHCI,IEDB
213183,NQKLIANQF,CASTPSPDTGELFF,SARS-CoV2,MHCI,IEDB
213184,NQKLIANQF,CSACRDRGYTEAFF,SARS-CoV2,MHCI,IEDB


In [94]:
McPAS = pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/McPAS_TCR/McPAS-TCR.csv",encoding='latin1')
McPAS['ID']= range(len(McPAS))
McPAS = McPAS[(McPAS['Species'] == 'Human')]
McPAS = McPAS[['CDR3.beta.aa','Pathology', 'Epitope.peptide','MHC','TRBV','TRBJ','ID']]
McPAS = McPAS.dropna(subset=['CDR3.beta.aa', 'Epitope.peptide','MHC'])
McPAS['database']='McPAS'
MHC=['HLA-A','HLA-B','HLA-C']
McPAS_cdr3 = McPAS[McPAS['MHC'].str.contains('|'.join(MHC))]
column_mapping = {
    "CDR3.beta.aa": "CDR3B",
    "Pathology": "antigen_species",
    "Epitope.peptide": "Epitope",
    "MHC": "MHC",
    "TRBV":"TRBV",
    "TRBJ": "TRBJ",
    "ID":"ID",
    'database':'database'}

McPAS_df=McPAS_cdr3.rename(columns=column_mapping)
McPAS_df

/tmp/ipykernel_43612/3564961936.py:1: DtypeWarning: Columns (13,17,24,26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  McPAS = pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/McPAS_TCR/McPAS-TCR.csv",encoding='latin1')


,CDR3B,antigen_species,Epitope,MHC,TRBV,TRBJ,ID,database
92,CASSLGNEQF,Melanoma,EAAGIGILTV,HLA-A*02,NaN,TRBJ2-1,92,McPAS
93,CASSLGVATGELF,Melanoma,EAAGIGILTV,HLA-A*02,NaN,TRBJ2-2,93,McPAS
94,CASSQEEGGGSWGNTIYF,Melanoma,EAAGIGILTV,HLA-A*02,NaN,TRBJ1-3,94,McPAS
95,CASSQEGLAGASQYF,Melanoma,EAAGIGILTV,HLA-A*02,NaN,TRBJ2-4,95,McPAS
96,CASSQETDIVFNOPQHF,Melanoma,EAAGIGILTV,HLA-A*02,NaN,TRBJ1-5,96,McPAS
...,...,...,...,...,...,...,...,...
39676,CSVGTGGTNEKLFF,Epstein Barr virus (EBV),GLCTLVAML,HLA-A*02:01,TRBV29-1,TRBJ5-1,39676,McPAS
39677,CASSEDGMNTEAFF,Epstein Barr virus (EBV),CLGGLLTMV,HLA-A*02:01,TRBV10-2*02,TRBJ21-1,39677,McPAS
39678,CASSYQGGNYGYTF,Epstein Barr virus (EBV),FLYALALLL,HLA-A*02:01,TRBV6-5*01,TRBJ17-1,39678,McPAS
39679,CASSKQGGNIQYF,Epstein Barr virus (EBV),FLYALALLL,HLA-A*02:01,TRBV6-5*01,TRBJ17-1,39679,McPAS


In [95]:
vdjdb = pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/VDJdb/vdjdb.txt", delimiter='\t', encoding='latin1')
vdjdb['ID']=range(len(vdjdb))
cdr3b = vdjdb[(vdjdb['gene'] == 'TRB') & (vdjdb['species'] == 'HomoSapiens') & (vdjdb['mhc.class'] == 'MHCI')]
#cdr3b = cdr3b[cdr3b['vdjdb.score'] != 0]
cdr3b['database']='vdjdb'
cdr3b = cdr3b[['cdr3', 'antigen.epitope', 'antigen.species','v.segm','j.segm','mhc.a','ID','database','vdjdb.score']]
column_mapping = {
    "cdr3": "CDR3B",
    "antigen.species": "antigen_species",
    "antigen.epitope": "Epitope",
    "mhc.a": "MHC",
    "v.segm":"TRBV",
    "j.segm": "TRBJ",
    "ID":"ID",
    'database':'database',
     'vdjdb.score':'vdjdb_score'}

vdjdb_df=cdr3b.rename(columns=column_mapping)
vdjdb_df

/tmp/ipykernel_43612/680379787.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cdr3b['database']='vdjdb'


,CDR3B,Epitope,antigen_species,TRBV,TRBJ,MHC,ID,database,vdjdb_score
1,CASSYLPGQGDHYSNQPQHF,FLKEKGGL,HIV-1,TRBV13*01,TRBJ1-5*01,HLA-B*08,1,vdjdb,2
2,CASSFEAGQGFFSNQPQHF,FLKEKGGL,HIV-1,TRBV13*01,TRBJ1-5*01,HLA-B*08,2,vdjdb,2
4,CASSFEPGQGFYSNQPQHF,FLKEKGGL,HIV-1,TRBV13*01,TRBJ1-5*01,HLA-B*08,4,vdjdb,2
6,CASSYEPGQVSHYSNQPQHF,FLKEKGGL,HIV-1,TRBV13*01,TRBJ1-5*01,HLA-B*08,6,vdjdb,2
8,CASSALASLNEQFF,FLKEKGGL,HIV-1,TRBV14*01,TRBJ2-1*01,HLA-B*08,8,vdjdb,2
...,...,...,...,...,...,...,...,...,...
92400,CASSQDRLVNADTQYF,RPGGKKHYM,HIV,TRBV4-2*01,TRBJ2-3*01,HLA-B*42:01,92400,vdjdb,1
92401,CASSLAGDRGNYGYTF,RPGGKKHYM,HIV,TRBV27*01,TRBJ1-2*01,HLA-B*42:01,92401,vdjdb,1
92402,CASRRQGALNTEAFF,RPGGKKHYM,HIV,TRBV27*01,TRBJ1-1*01,HLA-B*42:01,92402,vdjdb,1
92403,CASSQDRVVGADTQYF,RPGGKKHYM,HIV,TRBV4-2*01,TRBJ2-3*01,HLA-B*42:01,92403,vdjdb,0


# 4.Glanville_2017

In [96]:
import os
import csv
folder_path = "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/Glanville_2017/"
Glanville = pd.DataFrame()
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if filename.endswith(".tsv"):
        df = pd.read_csv(file_path, sep="\t")
        Glanville = pd.concat([Glanville, df])

Glanville['Epitope'] =Glanville['Epitope'].str.replace('.tsv', '')
Glanville['ID']=range(len(Glanville))
Glanville['database']='Glanville'
Glanville=Glanville[['aminoAcid','Epitope','Virus','HLA','database','ID']]

column_mapping = {
    "aminoAcid": "CDR3B",
    "Epitope": "Epitope",
    "database": "database",
    "Virus": "antigen_species",
    "HLA": "MHC",
    "Antigen":"Antigen_gene",
    'ID':'ID'
}

Glanville_df=Glanville.rename(columns=column_mapping)
Glanville_df

,CDR3B,Epitope,antigen_species,MHC,database,ID
0,CASRDRLVSYEQYF,VTEHDTLLY,CMV,HLA-A-*0101,Glanville,0
1,CASSWTGPSYEQYF,VTEHDTLLY,CMV,HLA-A-*0101,Glanville,1
2,CACRPRALYGYTF,VTEHDTLLY,CMV,HLA-A-*0101,Glanville,2
3,CAGTLAGGTDTQYF,VTEHDTLLY,CMV,HLA-A-*0101,Glanville,3
4,CAPRGGTNTGELFF,VTEHDTLLY,CMV,HLA-A-*0101,Glanville,4
...,...,...,...,...,...,...
734,CASSQSPGGQQFF,GLCTLVAML,EBV,HLA-A*0201,Glanville,2061
735,CASSQSPGGTQYF,GLCTLVAML,EBV,HLA-A*0201,Glanville,2062
736,CASSQTPGGTQYF,GLCTLVAML,EBV,HLA-A*0201,Glanville,2063
737,CASSSGTARYEQYF,GLCTLVAML,EBV,HLA-A*0201,Glanville,2064


# TBAdb

In [97]:
TBAdb=pd.read_excel("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/PIRD/TBAdb.xlsx")
TBAdb['database']='TBAdb'
TBAdb['ID']=range(len(TBAdb))
TBAdb

,ICDname,Disease.name,Category,Antigen,Antigen.sequence,HLA,Locus,CDR3.alpha.aa,CDR3.beta.aa,CDR3.alpha.nt,...,Evaluate.method,Case.num,Control.type,Control.num,Filteration,Journal,Pubmed.id,Grade,database,ID
0,A15,Tuberculosis,Pathogen,CFP10,TAAQAAVVRFQEAAN,DRB1*15:03,TRA-TRB,CIEHTNSGGSNYKLTF,CASSLEETQYF,-,...,Antigen-specific ex vivo proliferation,22,-,-,-,Nature,28636589,5,TBAdb,0
1,A15,Tuberculosis,Pathogen,CFP10,TAAQAAVVRFQEAAN,DRB1*15:03,TRA-TRB,CIVHTNSGGSNYKLTF,CASSPEETQYF,-,...,Antigen-specific ex vivo proliferation,22,-,-,-,Nature,28636589,5,TBAdb,1
2,A15,Tuberculosis,Pathogen,CFP10,TAAQAAVVRFQEAAN,DRB1*15:03,TRA-TRB,CIVKTNSGGSNYKLTF,CASSFEETQYF,-,...,Antigen-specific ex vivo proliferation,22,-,-,-,Nature,28636589,5,TBAdb,2
3,A15,Tuberculosis,Pathogen,ESAT-6;CFP-10,-,-,TRB,-,CASGRPYEQYF,-,...,Statistical analysis,25,Health,15,Stimulated with ESAT-6 or CFP-10,Tuberculosis,23845455,2,TBAdb,3
4,A15,Tuberculosis,Pathogen,ESAT-6;CFP-10,-,-,TRB,-,CASSFLERGLFFYEQYF,-,...,Statistical analysis,25,-,15,Stimulated with ESAT-6 or CFP-10,Tuberculosis,23845455,2,TBAdb,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51134,Z88,"Personal history of allergy to drugs, medicame...",Allergy,egg-white;ovomucoid,-,DRB1*08:02-DRB1*09:01/DRB4*01:01/DQA1*03:01/DQ...,TRA,CAVLFQGAQKLVF,-,-,...,Antigen-specific ex vivo proliferation,4,-,-,-,Clin Exp Allergy,12190663,5,TBAdb,51134
51135,Z88,"Personal history of allergy to drugs, medicame...",Allergy,egg-white;ovomucoid,-,DRB1*08:02-DRB1*09:01/DRB4*01:01/DQA1*03:01/DQ...,TRB,-,CASTRIRGGSDGSHF,-,...,Antigen-specific ex vivo proliferation,4,-,-,-,Clin Exp Allergy,12190663,5,TBAdb,51135
51136,Z88,"Personal history of allergy to drugs, medicame...",Allergy,egg-white;ovomucoid,-,DRB1*08:02-DRB1*09:01/DRB4*01:01/DQA1*03:01/DQ...,TRB,-,CATYSGELFF,-,...,Antigen-specific ex vivo proliferation,4,-,-,-,Clin Exp Allergy,12190663,5,TBAdb,51136
51137,Z88,"Personal history of allergy to drugs, medicame...",Allergy,egg-white;ovomucoid,-,DRB1*09:01-DRB1*14:03/DRB3*01:01/DRB4*01:01/DQ...,TRA,CVVNVGSASKIIF,-,-,...,Antigen-specific ex vivo proliferation,4,-,-,-,Clin Exp Allergy,12190663,5,TBAdb,51137


In [98]:
TBAdb = TBAdb[['Disease.name', 'Antigen.sequence', 'CDR3.beta.aa','HLA','Vbeta','Jbeta','ID','database']]
TBAdb = TBAdb[(TBAdb['CDR3.beta.aa'] != '-') & (TBAdb['Antigen.sequence'] != '-') & (TBAdb['HLA'] != '-')]
TBAdb = TBAdb.dropna(subset=['CDR3.beta.aa','Antigen.sequence','HLA'])
TBAdb_df = TBAdb[TBAdb['HLA'].str.startswith(('A*', 'B*', 'C*','HLAI'))]
TBAdb_df = TBAdb_df[~TBAdb_df['HLA'].str.startswith('HLAII')]
column_mapping = {
    "CDR3.beta.aa": "CDR3B",
    "Antigen.sequence": "Epitope",
    "Disease.name": "antigen_species",
    "HLA": "MHC",
    "Vbeta":"TRBV",
    "Jbeta": "TRBJ",
    "ID":"ID",
"database":"database"}
    
TBAdb_df=TBAdb_df.rename(columns=column_mapping)
TBAdb_df

,antigen_species,Epitope,CDR3B,MHC,TRBV,TRBJ,ID,database
2002,YellowFeverVirus,LLWNGPMAV,CASSQGTISYEQYF,A*02:01,TRBV14,TRBJ2-7,2002,TBAdb
2003,YellowFeverVirus,LLWNGPMAV,CASSVGGVTYNEQFF,A*02:01,TRBV9,TRBJ2-1,2003,TBAdb
2004,YellowFeverVirus,LLWNGPMAV,CASTDGGDSYTQYF,A*02:01,TRBV6-2,TRBJ2-3,2004,TBAdb
2005,YellowFeverVirus,LLWNGPMAV,CASTDGGDSYTQYF,A*02:01,TRBV6-3,TRBJ2-3,2005,TBAdb
2006,YellowFeverVirus,LLWNGPMAV,CATGAASSYEQYF,A*02:01,TRBV15,TRBJ2-7,2006,TBAdb
...,...,...,...,...,...,...,...,...
13800,InfluenzaA,GILGFVFTL,CASSIMGSGNTIYF,A*02:01,TRBV19*02,TRBJ1-3*01,13800,TBAdb
13801,InfluenzaA,GILGFVFTL,CASSIRSSYEQYF,A*02:01,TRBV1-1,TRBJ2-7*01,13801,TBAdb
13802,InfluenzaA,GILGFVFTL,CASSIRSTGELFF,A*02:01,TRBV19*02,TRBJ2-2*01,13802,TBAdb
13803,InfluenzaA,GILGFVFTL,CASSPLIGEQYF,A*02:01,TRBV2-1,TRBJ2-7*01,13803,TBAdb


In [99]:
filename = "/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/dbPepNeo2.0/MHC-I neoantigen-reactive TCRs.csv"
dbPepNeo = pd.read_csv(filename)
dbPepNeo['database']='dbPepNeo'
dbPepNeo['ID']=range(len(dbPepNeo))
dbPepNeo=dbPepNeo[['Cancer','Mut_peptide','CDR3','TCRBV','TCRBJ','database','ID']]
dbPepNeo = dbPepNeo[(dbPepNeo['Mut_peptide'] != '/') & (dbPepNeo['CDR3'] != '/')] 
column_mapping = {
    "CDR3": "CDR3B",
    "Mut_peptide": "Epitope",
    "Cancer": "antigen_species",
    "TCRBV":"TRBV",
    "TCRBJ": "TRBJ",
    "ID":"ID",
    "database":"database"}
dbPepNeo_df=dbPepNeo.rename(columns=column_mapping)
dbPepNeo_df

,antigen_species,Epitope,CDR3B,TRBV,TRBJ,database,ID
0,melanoma,CLNEYHLFL,CASSQDLSGGVYYGYTF,TRBV04-01*01,TRBJ01-02*01,dbPepNeo,0
1,melanoma,CLNEYHLFL,CSTLLAGGGDEQYV,TRBV29-01*01,TRBJ02-07*01,dbPepNeo,1
2,melanoma,CLNEYHLFL,CASSPTGLGETQYF,TRBV10-02*01,TRBJ02-05*01,dbPepNeo,2
3,melanoma,CLNEYHLFL,CSAPPGPLAHTQYF,/,TRBJ02-03*01,dbPepNeo,3
4,melanoma,CLNEYHLFL,CASSFGGPPNTGELFF,/,TRBJ02-02*01,dbPepNeo,4
...,...,...,...,...,...,...,...
398,melanoma,RVSTLRVSL,CASSLSWGSSYNEQFF,TRBV27,/,dbPepNeo,398
399,melanoma,SYMIMEIEL,CASRDHRVIYGYTF,TRBV6-5,/,dbPepNeo,399
400,melanoma,SYMIMEIEL,CASRDHRVIYGYTF,TRBV6-5,/,dbPepNeo,400
401,melanoma,SYMIMEIEL,CASSQGIAGSNQPQHF,TRBV4-1,/,dbPepNeo,401


In [100]:
MIRA = pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/MIRA/ImmuneCODE-MIRA-Release002/peptide-detail.csv")
MIRA1 = MIRA[MIRA["Amino Acids"].str.split(",").str.len() == 1]
MIRA1 = MIRA[MIRA["Amino Acids"].str.split(",").str.len() == 1]
MIRA1 = MIRA1.rename(columns={"Amino Acids": "Epitope"})
MIRA1[["CDR3B", "TRBV", "TRBJ"]] = MIRA1["TCR BioIdentity"].str.split("+", expand=True)
MIRA1 = MIRA1[MIRA1["TRBV"].str.match("TCRBV\d\d-\d\d")]
MIRA1 = MIRA1[MIRA1["TRBJ"].str.match("TCRBJ.+")]
MIRA1["TRBV"] = MIRA1["TRBV"].str.replace("C", "")
MIRA1["TRBJ"] =  MIRA1["TRBJ"].str.replace("C", "")
MIRA_df = MIRA1[MIRA1["CDR3B"].str.match("^C\w+F$")]
MIRA_df=MIRA_df[['Epitope','CDR3B','TRBV','TRBJ']]
MIRA_df['antigen_species']='SARS-CoV-2'
MIRA_df['database']='MIRA'
MIRA_df['MHC']='MHCI'
print(len(pd.unique(MIRA_df['Epitope'])))
MIRA_df['Epitope'].value_counts()
MIRA_df

148


,Epitope,CDR3B,TRBV,TRBJ,antigen_species,database,MHC
807,SEVGPEHSLAEY,CASSQVTAHTEAFF,TRBV04-01,TRBJ01-01,SARS-CoV-2,MIRA,MHCI
808,SEVGPEHSLAEY,CASSLSGDSYNEQFF,TRBV27-01,TRBJ02-01,SARS-CoV-2,MIRA,MHCI
809,SEVGPEHSLAEY,CASSQEVAGFMKQFF,TRBV04-01,TRBJ02-01,SARS-CoV-2,MIRA,MHCI
810,SEVGPEHSLAEY,CASTPGPSYEQYF,TRBV27-01,TRBJ02-07,SARS-CoV-2,MIRA,MHCI
811,SEVGPEHSLAEY,CASSLDIHNTEAFF,TRBV07-09,TRBJ01-01,SARS-CoV-2,MIRA,MHCI
...,...,...,...,...,...,...,...
128331,FSKQLQQSM,CASSAQPKSYEQYF,TRBV05-04,TRBJ02-07,SARS-CoV-2,MIRA,MHCI
128332,FSKQLQQSM,CASSHSGGAYNEQFF,TRBV05-04,TRBJ02-01,SARS-CoV-2,MIRA,MHCI
128333,FSKQLQQSM,CASSFSPGDPYGYTF,TRBV05-04,TRBJ01-02,SARS-CoV-2,MIRA,MHCI
128334,FSKQLQQSM,CASSSSSQDTQYF,TRBV05-04,TRBJ02-03,SARS-CoV-2,MIRA,MHCI


In [101]:
all_data=pd.concat([vdjdb_df,IEDB_df,dbPepNeo_df,TBAdb_df,Glanville_df,MIRA_df,McPAS_df], axis=0)
all_data = all_data[all_data['CDR3B'].str.match(r'^C.*F$')]
all_data['CDR3B_length'] = all_data['CDR3B'].apply(len) 
all_data['Epitope_length'] = all_data['Epitope'].apply(len)
condition = ((all_data['CDR3B'].isin(all_data[all_data['vdjdb_score'] == 0]['CDR3B'])) & (all_data['Epitope'].isin(all_data[all_data['vdjdb_score'] == 0]['Epitope'])))
deleted_data = all_data[condition]
all_data = all_data[~condition]
print(deleted_data['database'].value_counts())
non_standard_aa_pattern = r'[^ACDEFGHIKLMNPQRSTVWY]'
special_chars_pattern = r'[!@#$%^&*(),.?":{}|<>]'
lowercase_pattern = r'[a-z]'
all_data = all_data[~all_data['CDR3B'].str.contains(special_chars_pattern, regex=True, na=False)]
all_data = all_data[~all_data['CDR3B'].str.contains(lowercase_pattern, regex=True, na=False)]
all_data = all_data[~all_data['CDR3B'].str.contains(non_standard_aa_pattern, regex=True, na=False)]
all_data = all_data[~all_data['Epitope'].str.contains(special_chars_pattern, regex=True, na=False)]
all_data = all_data[~all_data['Epitope'].str.contains(lowercase_pattern, regex=True, na=False)]
all_data = all_data[~all_data['Epitope'].str.contains(non_standard_aa_pattern, regex=True, na=False)]
all_data.drop_duplicates(subset=['CDR3B', 'Epitope'], keep='first', inplace=True)
all_data

database
vdjdb        42238
IEDB         33053
TBAdb         4888
McPAS         3996
Glanville     1431
MIRA           845
dbPepNeo       210
Name: count, dtype: int64


,CDR3B,Epitope,antigen_species,TRBV,TRBJ,MHC,ID,database,vdjdb_score,CDR3B_length,Epitope_length
1,CASSYLPGQGDHYSNQPQHF,FLKEKGGL,HIV-1,TRBV13*01,TRBJ1-5*01,HLA-B*08,1.0,vdjdb,2.0,20,8
2,CASSFEAGQGFFSNQPQHF,FLKEKGGL,HIV-1,TRBV13*01,TRBJ1-5*01,HLA-B*08,2.0,vdjdb,2.0,19,8
4,CASSFEPGQGFYSNQPQHF,FLKEKGGL,HIV-1,TRBV13*01,TRBJ1-5*01,HLA-B*08,4.0,vdjdb,2.0,19,8
6,CASSYEPGQVSHYSNQPQHF,FLKEKGGL,HIV-1,TRBV13*01,TRBJ1-5*01,HLA-B*08,6.0,vdjdb,2.0,20,8
8,CASSALASLNEQFF,FLKEKGGL,HIV-1,TRBV14*01,TRBJ2-1*01,HLA-B*08,8.0,vdjdb,2.0,14,8
...,...,...,...,...,...,...,...,...,...,...,...
39029,CASSIFGGGLGEQFF,FLCMKALLL,Tumor associated antigen (TAA),TRBV19,TRBJ2-7,HLA-A2:01,39029.0,McPAS,NaN,15,9
39030,CALNGEISYNEQFF,FLCMKALLL,Tumor associated antigen (TAA),TRBV2,TRBJ2-2,HLA-A2:01,39030.0,McPAS,NaN,14,9
39031,CASSQGVNTGELFF,FLCMKALLL,Tumor associated antigen (TAA),TRBV4-2,TRBJ2-1,HLA-A2:01,39031.0,McPAS,NaN,14,9
39032,CSAETGLSYEQYF,FLCMKALLL,Tumor associated antigen (TAA),TRBV20-1,TRBJ2-7,HLA-A2:01,39032.0,McPAS,NaN,13,9


In [102]:
filtered_df = all_data[(all_data['CDR3B_length'].between(10, 18)) & (all_data['Epitope_length'] == 9)]
epitope_counts = filtered_df['Epitope'].value_counts()
filtered_df.loc[:, 'Epitope_Count'] = filtered_df['Epitope'].map(epitope_counts)
filtered_df = filtered_df.sort_values('Epitope')
filtered_df

/tmp/ipykernel_43612/51795249.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[:, 'Epitope_Count'] = filtered_df['Epitope'].map(epitope_counts)


,CDR3B,Epitope,antigen_species,TRBV,TRBJ,MHC,ID,database,vdjdb_score,CDR3B_length,Epitope_length,Epitope_Count
209963,CASSLSGWEFEQYF,AAASATLAL,NaN,NaN,NaN,MHCI,NaN,IEDB,NaN,14,9,3
209965,CASSQISNYGYTF,AAASATLAL,NaN,NaN,NaN,MHCI,NaN,IEDB,NaN,13,9,3
209967,CSAAPQMNTEAFF,AAASATLAL,NaN,NaN,NaN,MHCI,NaN,IEDB,NaN,13,9,3
21255,CASSQGFGANVLTF,AAFKRSCLK,MCPyV,TRBV19*01,TRBJ2-6*01,HLA-A*03:01,21255.0,vdjdb,3.0,14,9,5
21257,CASVSGQGVSPLHF,AAFKRSCLK,MCPyV,TRBV19*01,TRBJ1-6*01,HLA-A*03:01,21257.0,vdjdb,3.0,14,9,5
...,...,...,...,...,...,...,...,...,...,...,...,...
83976,CASSLEWSGYNEQFF,YYQLYSTQL,SARS-CoV-2,TRBV07-02,TRBJ02-01,MHCI,NaN,MIRA,NaN,15,9,16
83975,CASSLGGRVFDEQYF,YYQLYSTQL,SARS-CoV-2,TRBV03-01/03-02,TRBJ02-07,MHCI,NaN,MIRA,NaN,15,9,16
119100,CASSQVDPLWETQYF,YYQLYSTQL,SARS-CoV2,NaN,NaN,MHCI,NaN,IEDB,NaN,15,9,16
83973,CAWNHARQGAGELFF,YYQLYSTQL,SARS-CoV-2,TRBV30-01,TRBJ02-02,MHCI,NaN,MIRA,NaN,15,9,16


In [65]:
epitopes = ['GILGFVFTL', 'GLCTLVAML', 'NLVPMVATV']
three_df = filtered_df[filtered_df['Epitope'].isin(epitopes)]
three_df=three_df[['CDR3B','Epitope','antigen_species','database']]
three_df

,CDR3B,Epitope,antigen_species,database
208007,CASSSGTGSSYEQYF,GILGFVFTL,Influenza A virus,IEDB
208230,CASSTDSNTQYF,GILGFVFTL,Influenza A virus,IEDB
208234,CASSTFGTDTQYF,GILGFVFTL,Influenza A virus,IEDB
14859,CASGIIAGGYNEQF,GILGFVFTL,Influenza A virus,IEDB
208381,CASSTRSNTEAF,GILGFVFTL,Influenza A virus,IEDB
...,...,...,...,...
12246,CASRGTSNHEQF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB
12360,CASSSGLAGGPITGELF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB
202341,CASSYFSGPYEQYF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB
12253,CASSQDRVSANNEQF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB


In [66]:
unique_cdr3b_df1 = set(pos_combined['CDR3B'])
predition = three_df[~three_df['CDR3B'].isin(unique_cdr3b_df1)]
predition

,CDR3B,Epitope,antigen_species,database
208007,CASSSGTGSSYEQYF,GILGFVFTL,Influenza A virus,IEDB
208230,CASSTDSNTQYF,GILGFVFTL,Influenza A virus,IEDB
208234,CASSTFGTDTQYF,GILGFVFTL,Influenza A virus,IEDB
14859,CASGIIAGGYNEQF,GILGFVFTL,Influenza A virus,IEDB
208381,CASSTRSNTEAF,GILGFVFTL,Influenza A virus,IEDB
...,...,...,...,...
12368,CASSLGTEAF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB
12246,CASRGTSNHEQF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB
12360,CASSSGLAGGPITGELF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB
12253,CASSQDRVSANNEQF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB


In [67]:
predition.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/all_pos.csv")

# negative datasets collects

In [1]:
def outer_filter_negative_data(df,neg_tcr,ratio):
    unique_epitopes = df['Epitope'].unique()
    unique_tcrs = df['CDR3B'].unique()
    conversion_df = df[['Epitope', 'CDR3B']]
    positive_pairs = set([tuple(x) for x in conversion_df.to_numpy()])
    epitope_dist = df['Epitope'].value_counts()
    tcr_dist = df['CDR3B'].value_counts()
    epitope_freq_array = [epitope_dist[peptide] / len(df) for peptide in unique_epitopes]
    neg_pairs = set()
    neg_tcr_df = neg_tcr
 
    for i in range(len(unique_epitopes)):
        pairs_to_generate = round(epitope_dist[unique_epitopes[i]] * ratio)
        selected_tcrs = np.random.choice(neg_tcr_df['CDR3B'], size=pairs_to_generate, replace=False )
        new_pairs = [(unique_epitopes[i], tcr) for tcr in selected_tcrs]
        neg_pairs.update(new_pairs)
    negative_data = pd.DataFrame(neg_pairs, columns=['Epitope', 'CDR3B'])
    negative_data = negative_data.assign(Affinity=0)
    #df['Affinity'] = 1
    df.loc[:, 'Affinity'] = 1
    df = pd.concat([df, negative_data])
    df = df.reset_index(drop=True)
    return df


In [2]:
def filter(neg_data):
    # Filter data for TRB chain and cells marked as True
    neg_data = neg_data[(neg_data['chain'] == 'TRB')]
    neg_data.dropna(subset=['cdr3'], inplace=True)
    neg_data = neg_data[['cdr3']]
    neg_data.columns = ['CDR3B']
    non_standard_aa_pattern = r'[^ACDEFGHIKLMNPQRSTVWY]'
    special_chars_pattern = r'[!@#$%^&*(),.?":{}|<>]'
    lowercase_pattern = r'[a-z]'
    neg_data = neg_data[~neg_data['CDR3B'].str.contains(special_chars_pattern, regex=True, na=False)]
    neg_data = neg_data[~neg_data['CDR3B'].str.contains(lowercase_pattern, regex=True, na=False)]
    neg_data = neg_data[~neg_data['CDR3B'].str.contains(non_standard_aa_pattern, regex=True, na=False)]
    neg_data['cdr3_length'] = neg_data['CDR3B'].apply(len)
    neg_data = neg_data[(neg_data['cdr3_length'] >= 10) & (neg_data['cdr3_length'] <= 18)]
    neg_data = neg_data[neg_data['CDR3B'].str.startswith('C') & neg_data['CDR3B'].str.endswith('F')]
    neg_data = neg_data.drop_duplicates(subset=['CDR3B'], keep='first')
    return neg_data

In [9]:
import os
import pandas as pd
path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/raw_data/neg/"
os.chdir(path)
neg_data1=pd.read_csv('1_SC_Melanoma.csv')
neg_data2=pd.read_csv('2_nsclc.csv')
neg_data3=pd.read_csv('3_PBMC_TCR.csv')
neg_data4=pd.read_csv('4_health_donor4.csv')
neg_data5=pd.read_csv('5_PLL.csv')
neg_data6=pd.read_csv('6_nsclc.csv')
neg_data7=pd.read_csv('7_melanoma.csv')
neg_data8=pd.read_csv('8_PLL.csv')
neg_data9=pd.read_csv('9_melanoma.csv')


/tmp/ipykernel_42387/1471058651.py:12: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  neg_data8=pd.read_csv('8_PLL.csv')


In [10]:
len(neg_data18)+len(neg_data10)+len(neg_data11)+len(neg_data12)+len(neg_data13)+len(neg_data14)+len(neg_data15)+len(neg_data16)+len(neg_data17)

66397

In [8]:
neg_data10=pd.read_csv("/home/Rawdata/luyanping/raw_data/Suo_scRNA_TCR/data1/TCR/count/count5022/outs/filtered_contig_annotations.csv")
neg_data11=pd.read_csv("/home/Rawdata/luyanping/raw_data/Suo_scRNA_TCR/data1/TCR/count/count5023/outs/filtered_contig_annotations.csv")
neg_data12=pd.read_csv("/home/Rawdata/luyanping/raw_data/Suo_scRNA_TCR/data1/TCR/count/count5024/outs/filtered_contig_annotations.csv")
neg_data13=pd.read_csv("/home/Rawdata/luyanping/raw_data/Suo_scRNA_TCR/data1/TCR/count/count5025/outs/filtered_contig_annotations.csv")
neg_data14=pd.read_csv("/home/Rawdata/luyanping/raw_data/Suo_scRNA_TCR/data1/TCR/count/count5026/outs/filtered_contig_annotations.csv")
neg_data15=pd.read_csv("/home/Rawdata/luyanping/raw_data/Suo_scRNA_TCR/data1/TCR/count/count5027/outs/filtered_contig_annotations.csv")
neg_data16=pd.read_csv("/home/Rawdata/luyanping/raw_data/Suo_scRNA_TCR/data1/TCR/count/count5028/outs/filtered_contig_annotations.csv")
neg_data17=pd.read_csv("/home/Rawdata/luyanping/raw_data/Suo_scRNA_TCR/data1/TCR/count/count5029/outs/filtered_contig_annotations.csv")
neg_data18=pd.read_csv("/home/Rawdata/luyanping/raw_data/Suo_scRNA_TCR/data1/TCR/count/count5030/outs/filtered_contig_annotations.csv")


In [5]:
neg_data1=filter(neg_data1)
neg_data2=filter(neg_data2)
neg_data3=filter(neg_data3)
neg_data4=filter(neg_data4)
neg_data5=filter(neg_data5)
neg_data6=filter(neg_data6)
neg_data7=filter(neg_data7)
neg_data8=filter(neg_data8)
neg_data9=filter(neg_data9)
neg_data10=filter(neg_data10)
neg_data11=filter(neg_data11)
neg_data12=filter(neg_data12)
neg_data13=filter(neg_data13)
neg_data14=filter(neg_data14)
neg_data15=filter(neg_data15)
neg_data16=filter(neg_data16)
neg_data17=filter(neg_data17)
neg_data18=filter(neg_data18)

/tmp/ipykernel_43612/856945328.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_data.dropna(subset=['cdr3'], inplace=True)
/tmp/ipykernel_43612/856945328.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_data.dropna(subset=['cdr3'], inplace=True)
/tmp/ipykernel_43612/856945328.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_data.dropna(subset=['cdr3'], inplace=True)
/tmp/ipykernel_43612/856945328.py:4: SettingWithC

In [148]:
predition=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/all_pos.csv")
predition

,Unnamed: 0,CDR3B,Epitope,antigen_species,database
0,208007,CASSSGTGSSYEQYF,GILGFVFTL,Influenza A virus,IEDB
1,208230,CASSTDSNTQYF,GILGFVFTL,Influenza A virus,IEDB
2,208234,CASSTFGTDTQYF,GILGFVFTL,Influenza A virus,IEDB
3,14859,CASGIIAGGYNEQF,GILGFVFTL,Influenza A virus,IEDB
4,208381,CASSTRSNTEAF,GILGFVFTL,Influenza A virus,IEDB
...,...,...,...,...,...
6031,12368,CASSLGTEAF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB
6032,12246,CASRGTSNHEQF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB
6033,12360,CASSSGLAGGPITGELF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB
6034,12253,CASSQDRVSANNEQF,NLVPMVATV,Human herpesvirus 5 (Human cytomegalovirus),IEDB


In [6]:
ref_neg=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEINet/data/ref_neg_tcr.csv")
predition=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/all_pos.csv")

In [46]:
health=pd.concat([neg_data3, neg_data4], ignore_index=True)
health= health.drop_duplicates(subset=['CDR3B'], keep='first')
unique_cdr3b_df1 = set(ref_neg['CDR3.beta'])
health = health[~health['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(neg_combined['CDR3B'])
health = health[~health['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(predition['CDR3B'])
health = health[~health['CDR3B'].isin(unique_cdr3b_df1)]
health.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/health_neg.csv")
health

,CDR3B,cdr3_length
0,CASRDTLDVTETQYF,15
1,CASSLGALRIRNSPLHF,17
5,CSARRAGARGEAFF,14
6,CASSLGIEQFF,11
7,CASVQQSSTDTQYF,14
...,...,...
18583,CASSLEASGQNEQYF,15
18584,CASSLLANEQFF,12
18585,CASSQDRGEGVEAFF,15
18587,CASRFGAGIYEQYF,14


In [47]:
Diseases =pd.concat([neg_data1, neg_data2,neg_data5,neg_data6,neg_data7,neg_data8,neg_data9,neg_data10,neg_data11,neg_data12,neg_data13,neg_data14,neg_data15,neg_data16,neg_data17,neg_data18], ignore_index=True)
Diseases = Diseases.drop_duplicates(subset=['CDR3B'], keep='first')
unique_cdr3b_df1 = set(ref_neg['CDR3.beta'])
Diseases = Diseases[~Diseases['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(neg_combined['CDR3B'])
Diseases = Diseases[~Diseases['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(predition['CDR3B'])
Diseases = Diseases[~Diseases['CDR3B'].isin(unique_cdr3b_df1)]
Diseases.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/Diseases_neg.csv")
Diseases

,CDR3B,cdr3_length
1,CASSLEGGGAGEETQYF,17
10,CASNFNTGVEVNTQYF,16
11,CASSPGRTGYEGNTIYF,17
12,CSAQARLAGGPDTQYF,16
13,CASSPRPSGLYEQYF,15
...,...,...
28428,CASSLAGGLLREETQYF,17
28430,CASSRTGGYGTIYF,14
28431,CATSLTRTGGINEQFF,16
28432,CASSFRAVETGELFF,15


In [49]:
predict_data=outer_filter_negative_data(predition,health,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/1_health.csv")
predict_data=outer_filter_negative_data(predition,health,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/2_health.csv")
predict_data=outer_filter_negative_data(predition,health,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/3_health.csv")

predict_data=outer_filter_negative_data(predition,Diseases,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/1_Diseases.csv")
predict_data=outer_filter_negative_data(predition,Diseases,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/2_Diseases.csv")
predict_data=outer_filter_negative_data(predition,Diseases,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/3_Diseases.csv")


In [79]:
unique_cdr3b_df1 = set(predition['CDR3B'])
antigen_specificity = antigen_specificity[~antigen_specificity['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(neg_combined['CDR3B'])
antigen_specificity = antigen_specificity[~antigen_specificity['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(ref_neg['CDR3.beta'])
antigen_specificity = antigen_specificity[~antigen_specificity['CDR3B'].isin(unique_cdr3b_df1)]
antigen_specificity = antigen_specificity.drop_duplicates(subset=['CDR3B'], keep='first')
antigen_specificity

,CDR3B,Epitope,antigen_species,TRBV,TRBJ,MHC,ID,database,vdjdb_score,CDR3B_length,Epitope_length,Epitope_Count
209963,CASSLSGWEFEQYF,AAASATLAL,NaN,NaN,NaN,MHCI,NaN,IEDB,NaN,14,9,3
30827,CASSHAGLGF,AAGIGILTV,Homo sapiens (human),NaN,NaN,MHCI,NaN,IEDB,NaN,10,9,6
405,CASGDNSAETLYF,ACSRNRQYL,NaN,NaN,NaN,MHCI,NaN,IEDB,NaN,13,9,1
38461,CSHAGLGVEQF,AIGIGAYLV,Homo sapiens (human),NaN,NaN,MHCI,NaN,IEDB,NaN,11,9,1
202487,CSATTEGSSSGETQYF,AIMDKNIIL,Influenza A virus,NaN,NaN,MHCI,NaN,IEDB,NaN,16,9,3
...,...,...,...,...,...,...,...,...,...,...,...,...
83979,CASSVGPGTREQFF,YYQLYSTQL,SARS-CoV-2,TRBV09-01,TRBJ02-01,MHCI,NaN,MIRA,NaN,14,9,16
83975,CASSLGGRVFDEQYF,YYQLYSTQL,SARS-CoV-2,TRBV03-01/03-02,TRBJ02-07,MHCI,NaN,MIRA,NaN,15,9,16
119100,CASSQVDPLWETQYF,YYQLYSTQL,SARS-CoV2,NaN,NaN,MHCI,NaN,IEDB,NaN,15,9,16
83973,CAWNHARQGAGELFF,YYQLYSTQL,SARS-CoV-2,TRBV30-01,TRBJ02-02,MHCI,NaN,MIRA,NaN,15,9,16


In [84]:
predict_data=outer_filter_negative_data(predition,antigen_specificity,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/1_antigen_specificity.csv")
predict_data=outer_filter_negative_data(predition,antigen_specificity,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/2_antigen_specificity.csv")
predict_data=outer_filter_negative_data(predition,antigen_specificity,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/3_antigen_specificity.csv")

# 将数据集中C和F删除

In [87]:
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/neg_pos/"+i+'_'+j+'.csv'
            data=pd.read_csv(testfile_path)
            data['CDR3B'] = data['CDR3B'].apply(lambda x: x[1:-1])
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/three/no_CF/"+i+'_'+j+'.csv'
            data.to_csv(testfile_path)


# unseen 

In [92]:
import numpy as np
from functools import reduce
import pandas as pd

datasets = [DLpTCR, ERGOvdj_lstm, ERGOmc_lstm, ERGOvdj_ae, ERGOmc_ae, epiTCR,
            TEINet_Art, TEINet_large, ATM_TCR, AttnTAP_VDJdb, AttnTAP_McPAS,
            TPBTE_VDJdb, TPBTE_McPAS, SETE, TCRGP, TEIM, ImRex, NetTCR99]
unique_epitopes = [pd.unique(dataset['Epitope']) for dataset in datasets]
all_epitopes = reduce(np.union1d, unique_epitopes)
all_epitopes


array(['AAFKRSCLK', 'AAGIGILTV', 'AAKGRGAAL', ..., 'YYRARAGEAANF',
       'YYRRATRRIR', 'YYVGYLQPRTFLL'], dtype=object)

In [106]:
unseen = filtered_df[~filtered_df['Epitope'].isin(all_epitopes)]

In [ ]:
name_mapping = {'HomoSapiens': 'Unknown', 
                'lung_cancer': 'Lung cancer',
               'Homo sapiens (human)': 'Unknown', 
                'H1N1 subtype (H1N1)': 'InfluenzaA',
               'Influenza A virus (A/nt/60/1968(H3N2)) (Influenza A virus (A/NT/60/68/(H3N2)))': 'InfluenzaA', 
                'Influenza A virus': 'InfluenzaA',
               'Human herpesvirus 5 (Human cytomegalovirus)': 'CMV', 
                'Human coronavirus NL63 (Coronavirus NL63)': 'Coronavirus',
               'Mus musculus (mouse)': 'Unknown', 
                'SARS-CoV-2': 'SARS-CoV2',
               'Respirovirus muris (Murine respirovirus)': 'Unknown', 
                'Mammarenavirus choriomeningitidis (Lymphocytic choriomeningitis mammarenavirus)': 'LCMV',
               'synthetic': 'Unknown', 
                'Chlamydia trachomatis': 'C. trachomatis',
               'Influenza B virus (Influenza virus type B)': 'InfluenzaB', 
                'Human coronavirus 229E (Coronavirus 229E)': 'Coronavirus',
               'human papillomavirus 18 (Human papilloma virus type 18)': 'HPV', 
                'Toxoplasma gondii': 'Unknown',
               'Human papillomavirus 16 (Human papilloma virus type 16)': 'HPV', 
                'Influenza A virus (A/Japan/305/1957(H2N2)) (Influenza A virus (A/Japan/305/57(H2N2)))': 'InfluenzaA',
               'Influenza A virus (A/Puerto Rico/8/1934(H1N1)) (Influenza A virus (A/PR 8/34 (H1N1)))': 'InfluenzaA', 
                'Coxsackievirus B1 (Coxsackie B virus type 1)': 'CVB1',
               'Human coronavirus HKU1 (isolate N1):': 'Coronavirus', 
                'Human herpesvirus 4 (Epstein Barr virus)': 'EBV',
               'Human adenovirus 2 (Adenovirus 2)': 'Adenovirus', 
                'Salmonella enterica subsp. enterica serovar Typhi': 'S.Typhi',
                'Human coronavirus OC43 (Human coronavirus (strain OC43))': 'Coronavirus',
               'HIV1': 'HIV', 
                'Murine leukemia virus (murine leukemia virus MLV)': 'Unknown',
               'Simian immunodeficiency virus (Chimpanzee immunodeficiency virus)': 'Unknown', 
                'Human adenovirus 5 (Adenovirus 5)': 'Adenovirus',
               'Middle East respiratory syndrome-related coronavirus (MERS coronavirus):': 'MERS-CoV', 
                'glioblastoma': 'Glioblastoma',
               'Acidiphilium cryptum (Acidiphilum cryptum)': 'A.cryptum', 
                'melanoma': 'Melanoma',
               'HIV-1':'HIV',
               'Human coronavirus HKU1 (isolate N1)':'Coronavirus',
               'Middle East respiratory syndrome-related coronavirus (MERS coronavirus)':'Coronavirus',}

In [ ]:
unseen['antigen_species'] = unseen['antigen_species'].map(lambda x: name_mapping.get(x, x))
unseen['antigen_species'] = unseen['antigen_species'].fillna('Unknown')
pd.unique(unseen['antigen_species'])

In [9]:
unseen.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/Prediction/all/unseen/all_pos.csv")

In [107]:
health=pd.concat([neg_data3, neg_data4], ignore_index=True)
health= health.drop_duplicates(subset=['CDR3B'], keep='first')
unique_cdr3b_df1 = set(ref_neg['CDR3.beta'])
health = health[~health['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(neg_combined['CDR3B'])
health = health[~health['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(unseen['CDR3B'])
health = health[~health['CDR3B'].isin(unique_cdr3b_df1)]
health.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/health_neg.csv")
health

,CDR3B,cdr3_length
0,CASRDTLDVTETQYF,15
1,CASSLGALRIRNSPLHF,17
5,CSARRAGARGEAFF,14
6,CASSLGIEQFF,11
7,CASVQQSSTDTQYF,14
...,...,...
18583,CASSLEASGQNEQYF,15
18584,CASSLLANEQFF,12
18585,CASSQDRGEGVEAFF,15
18587,CASRFGAGIYEQYF,14


In [108]:
Diseases =pd.concat([neg_data1, neg_data2,neg_data5,neg_data6,neg_data7,neg_data8,neg_data9,neg_data10,neg_data11,neg_data12,neg_data13,neg_data14,neg_data15,neg_data16,neg_data17,neg_data18], ignore_index=True)
Diseases = Diseases.drop_duplicates(subset=['CDR3B'], keep='first')
unique_cdr3b_df1 = set(ref_neg['CDR3.beta'])
Diseases = Diseases[~Diseases['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(neg_combined['CDR3B'])
Diseases = Diseases[~Diseases['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(unseen['CDR3B'])
Diseases = Diseases[~Diseases['CDR3B'].isin(unique_cdr3b_df1)]
Diseases.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/Diseases_neg.csv")
Diseases

,CDR3B,cdr3_length
1,CASSLEGGGAGEETQYF,17
10,CASNFNTGVEVNTQYF,16
11,CASSPGRTGYEGNTIYF,17
12,CSAQARLAGGPDTQYF,16
13,CASSPRPSGLYEQYF,15
...,...,...
28428,CASSLAGGLLREETQYF,17
28430,CASSRTGGYGTIYF,14
28431,CATSLTRTGGINEQFF,16
28432,CASSFRAVETGELFF,15


In [109]:
unique_cdr3b_df1 = set(unseen['CDR3B'])
antigen_specificity = antigen_specificity[~antigen_specificity['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(neg_combined['CDR3B'])
antigen_specificity = antigen_specificity[~antigen_specificity['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(ref_neg['CDR3.beta'])
antigen_specificity = antigen_specificity[~antigen_specificity['CDR3B'].isin(unique_cdr3b_df1)]
antigen_specificity = antigen_specificity.drop_duplicates(subset=['CDR3B'], keep='first')
antigen_specificity

,CDR3B,Epitope,antigen_species,TRBV,TRBJ,MHC,ID,database,vdjdb_score,CDR3B_length,Epitope_length,Epitope_Count
30827,CASSHAGLGF,AAGIGILTV,Homo sapiens (human),NaN,NaN,MHCI,NaN,IEDB,NaN,10,9,6
38461,CSHAGLGVEQF,AIGIGAYLV,Homo sapiens (human),NaN,NaN,MHCI,NaN,IEDB,NaN,11,9,1
202487,CSATTEGSSSGETQYF,AIMDKNIIL,Influenza A virus,NaN,NaN,MHCI,NaN,IEDB,NaN,16,9,3
203562,CSVEELGPGYTF,AIMDKNIIL,Influenza A virus,NaN,NaN,MHCI,NaN,IEDB,NaN,12,9,3
200100,CAISESLGVDEQFF,AIMEKNIVL,Influenza A virus (A/California/07/2009(H1N1)),NaN,NaN,MHCI,NaN,IEDB,NaN,14,9,13
...,...,...,...,...,...,...,...,...,...,...,...,...
83971,CASSSSVGIHQPQHF,YYQLYSTQL,SARS-CoV-2,TRBV07-09,TRBJ01-05,MHCI,NaN,MIRA,NaN,15,9,16
83979,CASSVGPGTREQFF,YYQLYSTQL,SARS-CoV-2,TRBV09-01,TRBJ02-01,MHCI,NaN,MIRA,NaN,14,9,16
83975,CASSLGGRVFDEQYF,YYQLYSTQL,SARS-CoV-2,TRBV03-01/03-02,TRBJ02-07,MHCI,NaN,MIRA,NaN,15,9,16
119100,CASSQVDPLWETQYF,YYQLYSTQL,SARS-CoV2,NaN,NaN,MHCI,NaN,IEDB,NaN,15,9,16


In [110]:
predict_data=outer_filter_negative_data(unseen,health,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/1_health.csv")
predict_data=outer_filter_negative_data(unseen,health,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/2_health.csv")
predict_data=outer_filter_negative_data(unseen,health,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/3_health.csv")

predict_data=outer_filter_negative_data(unseen,Diseases,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/1_Diseases.csv")
predict_data=outer_filter_negative_data(unseen,Diseases,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/2_Diseases.csv")
predict_data=outer_filter_negative_data(unseen,Diseases,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/3_Diseases.csv")


predict_data=outer_filter_negative_data(unseen,antigen_specificity,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/1_antigen_specificity.csv")
predict_data=outer_filter_negative_data(unseen,antigen_specificity,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/2_antigen_specificity.csv")
predict_data=outer_filter_negative_data(unseen,antigen_specificity,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/3_antigen_specificity.csv")


/tmp/ipykernel_43612/3286755673.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Affinity'] = 1


In [111]:
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/CF/"+i+'_'+j+'.csv'
            data=pd.read_csv(testfile_path)
            data['CDR3B'] = data['CDR3B'].apply(lambda x: x[1:-1])
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/unseen/no_CF/"+i+'_'+j+'.csv'
            data.to_csv(testfile_path)


# 增加表位数量

In [133]:
import numpy as np
from functools import reduce
datasets = [ DLpTCR, ERGOvdj_lstm, ERGOmc_lstm, ERGOvdj_ae, ERGOmc_ae, epiTCR, 
    TEINet_Art, TEINet_large, ATM_TCR, AttnTAP_VDJdb, AttnTAP_McPAS, TEIM, ImRex]
unique_epitopes = [pd.unique(dataset['Epitope']) for dataset in datasets]
common_elements = reduce(np.intersect1d, unique_epitopes)
common_elements

array(['ATDALMTGY', 'FPRPWLHGL', 'GILGFVFTL', 'GLCTLVAML', 'LLWNGPMAV',
       'NLVPMVATV', 'RPRGEVRFL', 'VTEHDTLLY'], dtype=object)

In [134]:
def filter_dataframe(df):
    return df[df['Epitope'].isin(['ATDALMTGY', 'FPRPWLHGL', 'GILGFVFTL', 'GLCTLVAML', 'LLWNGPMAV','NLVPMVATV', 'VTEHDTLLY'])]
datasets = [ DLpTCR, ERGOvdj_lstm, ERGOmc_lstm, ERGOvdj_ae, ERGOmc_ae, epiTCR, 
    TEINet_Art, TEINet_large, ATM_TCR, AttnTAP_VDJdb, AttnTAP_McPAS, TEIM, ImRex]
filtered_dataframes = [filter_dataframe(df) for df in datasets]
combined_df = pd.concat(filtered_dataframes, ignore_index=True)

combined_df = combined_df.drop_duplicates()
pos_combined= combined_df[combined_df['Affinity'] == 1]
neg_combined = combined_df[combined_df['Affinity'] == 0]

In [135]:
epitopes = ['ATDALMTGY', 'FPRPWLHGL', 'GILGFVFTL', 'GLCTLVAML', 'LLWNGPMAV','NLVPMVATV', 'VTEHDTLLY']
seven_df = filtered_df[filtered_df['Epitope'].isin(epitopes)]
seven_df=seven_df[['CDR3B','Epitope','antigen_species','database']]
seven_df

,CDR3B,Epitope,antigen_species,database
78700,CASSQSGQLDTQYF,ATDALMTGY,HCV,vdjdb
78701,CASSSLGQLETQYF,ATDALMTGY,HCV,vdjdb
78702,CASSLLGQLYEQYF,ATDALMTGY,HCV,vdjdb
78703,CASNPAGQLETQYF,ATDALMTGY,HCV,vdjdb
78645,CASSPREIGNQPQHF,ATDALMTGY,HCV,vdjdb
...,...,...,...,...
200906,CASSLAPTSGGELF,VTEHDTLLY,Human herpesvirus 5 (Human cytomegalovirus),IEDB
23158,CASSAIGGAYEQYF,VTEHDTLLY,Human herpesvirus 5 (Human cytomegalovirus),IEDB
23148,CASSWTGGGARDTQYF,VTEHDTLLY,Human herpesvirus 5 (Human cytomegalovirus),IEDB
23146,CASSWSGGGTTDTQYF,VTEHDTLLY,Human herpesvirus 5 (Human cytomegalovirus),IEDB


In [136]:
seven_df['Epitope'].value_counts()

Epitope
GLCTLVAML    6297
GILGFVFTL    3818
NLVPMVATV    2596
LLWNGPMAV    1930
ATDALMTGY     156
VTEHDTLLY      91
FPRPWLHGL      52
Name: count, dtype: int64

In [127]:
unique_cdr3b_df1 = set(pos_combined['CDR3B'])
predition = seven_df[~seven_df['CDR3B'].isin(unique_cdr3b_df1)]
predition

,CDR3B,Epitope,antigen_species,database
19700,CASSGGLINTGELF,ATDALMTGY,Hepatitis C virus,IEDB
19690,CASSLRTAGYNEQF,ATDALMTGY,Hepatitis C virus,IEDB
19691,CASSGTRQISGTEAF,ATDALMTGY,Hepatitis C virus,IEDB
19694,CASSPWTSGGAYNEQF,ATDALMTGY,Hepatitis C virus,IEDB
30746,CASSLGLAGGEETQYYF,ATDALMTGY,Hepatitis C virus,IEDB
...,...,...,...,...
211032,CAISDRTTSEGQPQHF,VTEHDTLLY,Human herpesvirus 5 (Human cytomegalovirus),IEDB
211481,CASSVGAGPSYEQYF,VTEHDTLLY,Human herpesvirus 5 (Human cytomegalovirus),IEDB
211500,CATSRATGGRTGELFF,VTEHDTLLY,Human herpesvirus 5 (Human cytomegalovirus),IEDB
200907,CASSRTGVGGRSYEQYVF,VTEHDTLLY,Human herpesvirus 5 (Human cytomegalovirus),IEDB


In [128]:
predition['Epitope'].value_counts()

Epitope
GILGFVFTL    3298
NLVPMVATV    2020
GLCTLVAML     719
LLWNGPMAV     227
ATDALMTGY      13
VTEHDTLLY       6
Name: count, dtype: int64

In [129]:
predition.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/all_pos.csv")
ref_neg=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/TEINet/data/ref_neg_tcr.csv")


In [150]:
predition=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/all_pos.csv")
pd.unique(predition['database'])

array(['IEDB', 'vdjdb', 'TBAdb'], dtype=object)

In [130]:
health=pd.concat([neg_data3, neg_data4], ignore_index=True)
health= health.drop_duplicates(subset=['CDR3B'], keep='first')
unique_cdr3b_df1 = set(ref_neg['CDR3.beta'])
health = health[~health['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(neg_combined['CDR3B'])
health = health[~health['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(predition['CDR3B'])
health = health[~health['CDR3B'].isin(unique_cdr3b_df1)]
health.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/health_neg.csv")
print(len(health))


Diseases =pd.concat([neg_data1, neg_data2,neg_data5,neg_data6,neg_data7,neg_data8,neg_data9,neg_data10,neg_data11,neg_data12,neg_data13,neg_data14,neg_data15,neg_data16,neg_data17,neg_data18], ignore_index=True)
Diseases = Diseases.drop_duplicates(subset=['CDR3B'], keep='first')
unique_cdr3b_df1 = set(ref_neg['CDR3.beta'])
Diseases = Diseases[~Diseases['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(neg_combined['CDR3B'])
Diseases = Diseases[~Diseases['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(predition['CDR3B'])
Diseases = Diseases[~Diseases['CDR3B'].isin(unique_cdr3b_df1)]
Diseases.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/Diseases_neg.csv")
print(len(Diseases))

unique_cdr3b_df1 = set(predition['CDR3B'])
antigen_specificity = antigen_specificity[~antigen_specificity['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(neg_combined['CDR3B'])
antigen_specificity = antigen_specificity[~antigen_specificity['CDR3B'].isin(unique_cdr3b_df1)]
unique_cdr3b_df1 = set(ref_neg['CDR3.beta'])
antigen_specificity = antigen_specificity[~antigen_specificity['CDR3B'].isin(unique_cdr3b_df1)]
antigen_specificity = antigen_specificity.drop_duplicates(subset=['CDR3B'], keep='first')
antigen_specificity.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/antigen_specificity_neg.csv")

print(len(antigen_specificity))

8210
10327
35903


In [131]:
predict_data=outer_filter_negative_data(predition,health,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/1_health.csv")
predict_data=outer_filter_negative_data(predition,health,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/2_health.csv")
predict_data=outer_filter_negative_data(predition,health,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/3_health.csv")

predict_data=outer_filter_negative_data(predition,Diseases,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/1_Diseases.csv")
predict_data=outer_filter_negative_data(predition,Diseases,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/2_Diseases.csv")
predict_data=outer_filter_negative_data(predition,Diseases,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/3_Diseases.csv")


predict_data=outer_filter_negative_data(predition,antigen_specificity,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/1_antigen_specificity.csv")
predict_data=outer_filter_negative_data(predition,antigen_specificity,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/2_antigen_specificity.csv")
predict_data=outer_filter_negative_data(predition,antigen_specificity,1)
predict_data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/3_antigen_specificity.csv")

/tmp/ipykernel_43612/3286755673.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'Affinity'] = 1


In [132]:
name=['1','2','3']
me=['health','Diseases','antigen_specificity']
for i in name:
    for j in me:
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/neg_pos/"+i+'_'+j+'.csv'
            data=pd.read_csv(testfile_path)
            data['CDR3B'] = data['CDR3B'].apply(lambda x: x[1:-1])
            testfile_path="/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/database/benchmark_dataset/10cross_validation/Prediction/all/seen/seven/no_CF/"+i+'_'+j+'.csv'
            data.to_csv(testfile_path)


In [143]:
def negative_data(df,ratio):
    def random_recombination(df, epitope_dist, tcr_dist, ratio):
        unique_epitopes = df['Epitope'].unique()
        unique_tcrs = df['CDR3B'].unique()
        conversion_df = df[['Epitope', 'CDR3B']]
        positive_pairs = set([tuple(x) for x in conversion_df.to_numpy()])
        epitope_freq_array = [epitope_dist[peptide] / len(df) for peptide in unique_epitopes]
        tcr_freq_array = [tcr_dist[tcr] / len(df) for tcr in unique_tcrs]    
        neg_pairs = set()
        for pep in unique_epitopes:
            i = 0
            pairs_to_generate = round(epitope_dist[pep] * ratio)
            while i < pairs_to_generate:
                tcr = np.random.choice(unique_tcrs, p=tcr_freq_array)
                pair = (pep, tcr)
                if pair not in positive_pairs and pair not in neg_pairs:
                    neg_pairs.add(pair)
                    i += 1        
        negative_data = pd.DataFrame(neg_pairs, columns = ['Epitope', 'CDR3B'])
        negative_data = negative_data.assign(Affinity=0)
        return negative_data

    # Ratio of negative to positive data
    df['Affinity']=1
    epitope_distribution = df['Epitope'].value_counts()
    tcr_distribution = df['CDR3B'].value_counts()
    negative_data = random_recombination(df, epitope_distribution, tcr_distribution, ratio)
    df = pd.concat([df, negative_data])
    df = df.reset_index(drop=True)
    return df

In [145]:
data=pd.read_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/SETE/data/epitope_tcr_pairs.csv")
data=negative_data(data,1)
data.to_csv("/home/luyanping/data/TCR_epitope_prediction/Compare_models_same_data/SETE/data/train_raw_data.csv")